# TODO: 
- align methodology (col names etc etc) exactly to new data gathering process

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine  
import datetime 
import yfinance as yf
import warnings
# import norgatedata
warnings.filterwarnings('ignore')

# load metadata and relevant data
sharadar_sectors = pd.read_csv("C:/Users/lbianculli/backup/quant_analysis/dev/data/maps/sharadar_meta2.csv")
price_data = pd.read_csv("C:/Users/lbianculli/dev/us_equities/norgate/nogate_pricing.csv").drop(["Unnamed: 0"], axis=1)
all_macro_data = pd.read_csv("C:/Users/lbianculli/dev/us_equities/macro/macro_data.csv").drop(["Unnamed: 0"], axis=1)

# fundamental data
all_fundamental_data = pd.read_csv("C:/Users/lbianculli/dev/us_equities/ML_categorization/quandl_fundamentals.csv")
all_fundamental_data = all_fundamental_data.loc[all_fundamental_data["dimension"] == "MRQ"]

for c in all_fundamental_data.columns:
    try:
        all_fundamental_data[c] = all_fundamental_data[c].astype(float)
    except Exception as e:
        pass


In [2]:
# get nearest business day so we can merge later
from pandas.tseries.offsets import BDay

price_data["date_dt"] = pd.to_datetime(price_data["date_dt"], format='%Y-%m-%d')
all_fundamental_data["date_dt"] = pd.to_datetime(all_fundamental_data["calendar_date"], format='%Y-%m-%d')
all_fundamental_data["concat"] = all_fundamental_data["ticker_symbol"] + "-" + all_fundamental_data["date_dt"].dt.strftime(date_format="%Y-%m-%d")
all_fundamental_data["ticker_symbol"] = all_fundamental_data["ticker_symbol"].str.replace(".", "-")  # YF idiosyncracy

# get largest companies according to average market cap
n_largest = 1000
grouped_data = all_fundamental_data.groupby("ticker_symbol").mean()
largest_company_data = grouped_data.sort_values("market_capitalization", ascending=False).head(n_largest)
largest_companies = largest_company_data.index.unique()
largest_companies = [c.replace("-", ".") for c in largest_companies]

# get fundamentals of all largest companies. Aggregation above would have dropped some columns
largest_company_data = all_fundamental_data.loc[all_fundamental_data["ticker_symbol"].isin(largest_companies)]

In [3]:
def tp_tag(row):
    """
    0: rebound
    1: bull
    2: bear
    3: contraction
    """
    if (row["fast_tag"] > 0) & (row["slow_tag"] < 0):
        return 0 
    
    if (row["fast_tag"] > 0) & (row["slow_tag"] > 0):
        return 1 
    
    if (row["fast_tag"] < 0) & (row["slow_tag"] < 0):
        return 2 
    
    if (row["fast_tag"] < 0) & (row["slow_tag"] > 0):
        return 3

In [4]:
# TODO: Add in the 
equity_data = {}

macro_columns = ['unemp_ma_diff', 'confidence', 'MICH', 'bus_credit_change', 'con_credit_change', 
                 'pce_change', 'Adj Close', 'vix_change', 'inverted']

# # norgate params
# price_adjust = norgatedata.StockPriceAdjustmentType.TOTALRETURN 
# padding_setting = norgatedata.PaddingType.NONE   
# start_date = '2020-10-01'  # should change to look up off dataset
# ts_format = 'pandas-dataframe'

for ticker in largest_companies:
    company_data = all_fundamental_data.loc[all_fundamental_data["ticker_symbol"] == ticker]

    if ticker[-1] == "1":
        ticker = ticker[:-1]
        
    if ticker not in price_data["ticker"].unique():
        ticker_obj = yf.Ticker(ticker)
        company_price_data = ticker_obj.history(start='2018-09-01', end='2021-10-01').reset_index()
        
        try:
            company_price_data.columns = ["date_dt", "open", "high", "low", "close", "volume", "div", "splits"]
        except Exception as e:  # occurs if no data
            company_price_data.columns = ["date_dt", "open", "high", "low", "close1", "close", "volume"]
        
        company_price_data["ticker"] = ticker
        
    else:
        company_price_data = price_data.loc[price_data["ticker"] == ticker]

    if company_price_data.shape[0] > 0:
        ### Price Data
        # first, get updated data that we can use to merge with historical data
        try:
            # get columns to align with historical data
            company_price_data["date"] = company_price_data["date_dt"].dt.strftime(date_format="%Y-%m-%d")
            company_price_data["year"] = company_price_data["date_dt"].dt.year
            company_price_data["month"] = company_price_data["date_dt"].dt.month
            company_price_data["ticker"] = ticker
            company_price_data["period"] = company_price_data["year"].astype(str) + "-" + company_price_data["month"].astype(str)
            company_price_data["id"] = company_price_data["ticker"] + "-" + company_price_data["period"]

            # merge
            company_price_data = pd.concat([company_price_data, company_price_data])

        except ValueError as e:
    #         print(f"No new price data available for {ticker}")
            pass

        # get month and year end date for calculating returns
        company_price_data["day"] = company_price_data["date_dt"].dt.day
        company_price_data["day_tag"] = company_price_data["day"].shift(-1) - company_price_data["day"]
        company_price_data["day_tag"] = company_price_data["day_tag"].replace(np.nan, -1)  # in case we get between financial release and end of month

        # get returns and volatility
        company_price_data["rets"] = company_price_data["close"] / company_price_data["close"].shift(1) - 1
        company_price_data["log_rets"] = np.log(1 + company_price_data["rets"])
        company_price_data["volatility"] = company_price_data["log_rets"].rolling(252).std()
        company_price_data["1mo_rets"] = company_price_data["close"] / company_price_data["close"].shift(21) - 1
        company_price_data["1yr_rets"] = company_price_data["close"] / company_price_data["close"].shift(252) - 1

        # compute log and forward returns
        company_price_data["1mo_log_rets"] = np.log(1 + company_price_data["1mo_rets"])
        company_price_data["1yr_log_rets"] = np.log(1 + company_price_data["1yr_rets"])

        company_price_data["1mo_fwd_rets"] = company_price_data["close"].shift(-21) / company_price_data["close"] - 1
        company_price_data["1mo_fwd_log_rets"] = np.log(1 + company_price_data["1mo_fwd_rets"])

        # limit to month-end data to align with fundaments
        monthly_data = company_price_data.loc[company_price_data["day_tag"] < 0]

        # get tags for momentum windows
        try:
            monthly_data["fast_tag"] = np.where(monthly_data["1mo_rets"] < 0, -1, 1) 
            monthly_data["slow_tag"] = np.where(monthly_data["1yr_rets"] < 0, -1, 1)
            monthly_data[["1mo_rets", "1yr_rets", "slow_tag", "fast_tag"]].head()
            monthly_data["tp_tag"] = monthly_data.apply(lambda x: tp_tag(x), axis=1)

            # get turning point, defined as disagreement between slow and fast signals (does this make sense??)
            monthly_data["tp"] = np.where(monthly_data["slow_tag"] != monthly_data["fast_tag"], 1, 0)
            monthly_data["tp_last_yr"] = monthly_data["tp"].rolling(12).sum()

        except Exception as e:
            pass  # no data

        ### Fundamental data
        # and get other fundamental data as well
        company_data = company_data.sort_values("calendar_date")
        
        # create standard deviation (variability calcs)
        company_data["fcf_std"] = company_data["free_cash_flow_per_share"].iloc[-4:].std() 
        company_data["capex_std"] = company_data["capital_expenditure"].iloc[-4:].std()
        company_data["earnings_std"] = company_data['earnings_per_basic_share_usd'].iloc[-4:].std()
        company_data["earnings_std"] = company_data["earnings_std"].replace(np.inf, np.nan)
        company_data["cfo_std"] = company_data['net_cash_flow_from_operations'].iloc[-4:].std()
        company_data["cfo_std"] = company_data["cfo_std"].replace(np.inf, np.nan)

        company_data["avg_assets"] = (company_data["total_assets"] + company_data["total_assets"].shift(1)) / 2
        company_data["avg_equity"] = (company_data["shareholders_equity"] + company_data["shareholders_equity"].shift(1)) / 2
        company_data["avg_wc"] = (company_data["working_capital"] + company_data["working_capital"].shift(1)) / 2
        company_data["avg_debt"] = (company_data["total_debt"] + company_data["total_debt"].shift(1)) / 2
        
        # setup for targets later
        company_data["fwd_fcf_per_share"] = company_data["free_cash_flow_per_share"].shift(-1)
#         company_data["fwd_eps"] = company_data["earnings_per_basic_share_usd"].shift(-1)

        # replace with 0 -- it is likely four straight 0 periods
        company_data["capex_std"] = company_data["capex_std"].replace(np.inf, 0.0)

        # for calculations that resulted in Nans, impute with median. should this be done later?
        company_data["capex_std"] = company_data["capex_std"].fillna(company_data["capex_std"].median())
        company_data["earnings_std"] = company_data["capex_std"].fillna(company_data["earnings_std"].median())
        company_data["avg_assets"] = company_data["avg_assets"].fillna(company_data["avg_assets"].median())
        company_data["avg_equity"] = company_data["avg_equity"].fillna(company_data["avg_equity"].median())
        company_data["avg_wc"] = company_data["avg_wc"].fillna(company_data["avg_wc"].median())
        company_data["avg_debt"] = company_data["avg_debt"].fillna(company_data["avg_debt"].median())
        company_data["avg_capital"] = company_data["avg_debt"] + company_data["avg_equity"]

        # get year, month, day
        company_data["year"] = company_data["date_dt"].dt.year
        company_data["month"] = company_data["date_dt"].dt.month
        company_data["day"] = company_data["date_dt"].dt.day

        # convert to strings for ID
        company_data["year"] = company_data["year"].astype(str)
        company_data["month"] = company_data["month"].astype(str)
        company_data["day"] = company_data["day"].astype(str)

        # get ID for merge
        company_data["period"] = company_data["year"] + "-" + company_data["month"]
        company_data["concat"] = company_data["ticker_symbol"] + "-" + company_data["period"]

        merged_data_final = monthly_data.merge(company_data, left_on="id", right_on="concat", how="outer")
    
#         macro_data = all_macro_data
#         macro_data["ticker"] = ticker
#         macro_data["id"] = macro_data["ticker"] + "-" + macro_data["date"]
#         merged_data_final = merged_data.merge(macro_data, left_on="id", right_on="id", how="outer")

        # just ffill as we only need quarterly values and this will avoid look-ahead bias
#         for c in macro_columns:
#             merged_data_final[c] = merged_data_final[c].fillna(method="ffill")

        equity_data[ticker] = merged_data_final.dropna(subset=["volatility"])


- BRK.B: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted
- MOB: No data found for this date range, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted
- WYE: No data found for this date range, symbol may be delisted
- TFCF: No data found, symbol may be delisted
- WLA: No data found for this date range, symbol may be delisted
- TYC: No data found for this date range, symbol may be delisted
- CPQ: No data found for this date range, symbol may be delisted
- KRFT: No data found for this date range, symbol may be delisted
- USW: No data found for this date range, symbol may be delisted
- JAVA: Data doesn't exist for startDate = 1535785200, endDate = 1633071600
- CELG: No data found, symbol may be delisted
- FBF: No data found for this date range, symbol may be delisted


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
# probably merge macro data after everything else is done
equity_df = pd.concat([equity_data[t] for t in equity_data])
equity_df = equity_df.dropna(subset=["concat"])
equity_df["quarter"] = equity_df["date_dt_x"].dt.quarter
equity_df["quarter_end"] = equity_df["quarter"] % 3
equity_df = equity_df.merge(sharadar_sectors[["ticker", "sicsector"]], left_on="ticker_symbol", right_on="ticker")

# drop any duplicates
equity_df = equity_df[~equity_df["id"].duplicated()]

equity_df.shape


In [ ]:
# write to CSV
equity_df.to_csv("C:/Users/lbianculli/dev/us_equities/ML_categorization/all_fundamentals.csv")


### final_preprocessing file next